In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None
    
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [2]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
table_suffix = ["0"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  b.mw_file_suffix as mw_file_size,
  b.mw_file_prefix as mw_file_directory,
  b.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from (
       select
         mw_file_hash,
         section_name,
         pointerto_raw_data,
         virtual_size,
         count(1) as cnt
       from mw_index_2017_section_%s
       where CNT_CODE = 1 and MEM_EXECUTE = 1
       group by mw_file_hash) a
  inner join mw_index_2017_%s b on a.mw_file_hash = b.mw_file_hash and a.cnt = 1 and b.mw_num_engines <> -1 and
                                  (b.mw_num_engines >= 4 or b.mw_num_engines = 0) and
                                  b.mw_file_prefix in ('201704')
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 9.1742422580719 seconds ---
5899


In [3]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
table_suffix = ["0"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  b.mw_file_suffix as mw_file_size,
  b.mw_file_prefix as mw_file_directory,
  b.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from (
       select
         mw_file_hash,
         section_name,
         pointerto_raw_data,
         virtual_size,
         count(1) as cnt
       from mw_index_2017_section_%s
       where CNT_CODE = 1 and MEM_EXECUTE = 1
       group by mw_file_hash) a
  inner join mw_index_2017_%s b on a.mw_file_hash = b.mw_file_hash and a.cnt = 1 and b.mw_num_engines <> -1 and
                                  (b.mw_num_engines >= 4 or b.mw_num_engines = 0) and a.virtual_size < 10240 and
                                  b.mw_file_prefix in ('201705')
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

--- 4.925442695617676 seconds ---
945


In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 10240

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines >= 4 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines >= 4 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

In [7]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, labels, batch_size=32, dim=8192, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        np.random.seed(5432)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)
        y = np.zeros(self.batch_size, dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > max_length:
                bytes_data = [int(single_byte) for single_byte in in_file.read(max_length)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data
            y[i] = self.labels[ID]

        return X, y

In [ ]:
# import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Multiply, GlobalMaxPooling1D, concatenate, Dropout, Maximum, Flatten, Activation
from keras.models import load_model
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

from keras.models import Model

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

max_len = max_length
time = time.time()

summary = {
    'batch_size': 16,
    'epochs': 64,
}

def get_p(key):
    return summary[key]

def res_dilated_block_1(filters, dilation_rate, filter_size):
    def layer(input_):
        
        input_ = Activation('relu')(input_)
        conv_1 = Conv1D(filters, filter_size, padding='same')(input_)
        conv_2 = Conv1D(filters, filter_size, dilation_rate=dilation_rate, padding='causal',
                        activation='sigmoid')(input_)
        
        merged = Multiply()([conv_1, conv_2])
        if dilation_rate > 1:
            out = Maximum()([merged, input_])
        else:
            out = merged
        return out, conv_1
    return layer

def get_model(max_features):

    input = Input(shape=(max_len,))
    seq = Embedding(128, 8, input_length=max_len)(input)
    res_layers = []
    for i in range(9):
        seq, curr_layer = res_dilated_block_1(32, 2**i, 256//i if i > 0 else 256)(seq)
        res_layers.append(curr_layer)
    
    seq = GlobalMaxPooling1D()(seq)
    seq = Dense(1, activation='sigmoid')(seq)
    model = Model(input=input, output=seq)

    model.summary()
    from keras.utils.vis_utils import plot_model
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    return model

if __name__ == '__main__':
    batch_size = get_p("batch_size")
    epochs = get_p("epochs")

    model = get_model(1024)

    print('Length of the train: ', len(x_train))
    print('Length of the validation: ', len(x_val))

    #         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
    file_path = "/home/zhaoqi/BaseTrain/models/"+ str(time) +"-{epoch:04d}-{val_loss:.5f}-{val_acc:.5f}.h5"
    early_stopping = EarlyStopping("val_loss", patience=3, verbose=0, mode='auto')
    check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')
    callbacks_list = [check_point, early_stopping]

    # Generators
    training_generator = DataGenerator(range(len(x_train)), x_train, y_train, batch_size, max_len)
    validation_generator = DataGenerator(range(len(x_val)), x_val, y_val, batch_size, max_len)

    model.compile(loss='binary_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])
    
    model.fit_generator(generator=training_generator,
                             validation_data=validation_generator,
                             use_multiprocessing=True,
                             epochs=epochs,
                             workers=6,
                             callbacks=callbacks_list)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 10240)        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 10240, 8)     1024        input_2[0][0]                    
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 10240, 8)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_19 (Conv1D)              (None, 10240, 32)    65568       activation_10[0][0]              
__________________________________________________________________________________________________
conv1d_20 

Length of the train:  5309
Length of the validation:  590
Epoch 1/64
331/331 [==============================] - 118s 355ms/step - loss: 0.4652 - acc: 0.7672 - val_loss: 0.4132 - val_acc: 0.8212

Epoch 00001: saving model to /home/zhaoqi/BaseTrain/models/1533013817.1850631-0001-0.41321-0.82118.h5
Epoch 2/64
331/331 [==============================] - 114s 344ms/step - loss: 0.3569 - acc: 0.8454 - val_loss: 0.4495 - val_acc: 0.8160

Epoch 00002: saving model to /home/zhaoqi/BaseTrain/models/1533013817.1850631-0002-0.44948-0.81597.h5
Epoch 3/64
  3/331 [..............................] - ETA: 1:49 - loss: 0.2046 - acc: 0.9375

In [40]:
model_dir = '/home/zhaoqi/BaseTrain/models/'
f_name = '1532956752.1549792-0005-0.10709-0.96727.h5'
c_model = load_model(model_dir + f_name)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, batch_size, max_length, False)
y_pred = model.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

904/904 [==============================] - 88s 97ms/step


In [11]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
# import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

In [47]:
estimate_model(y_pred, y_test[0:len(y_pred)])

loss : nan
auc score : 0.90337
accuracy score : 0.86634
thre: 0.9999341965
fp:  0.0009204148
recall:  0.4297299155


/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


(0.9033727002444281, nan, 0.42972991546972716)

In [49]:
estimate_model(y_pred, y_test[0:len(y_pred)])

loss : nan
auc score : 0.90337
accuracy score : 0.86634
thre: 0.9999341965
fp:  0.0009204148
recall:  0.4297299155


/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


(0.9033727002444281, nan, 0.42972991546972716)